<span type="title">异常和中断</span> | <span type="update">2018-07-18</span> | <span type="version">2</span>

<span type="intro"><p type="card-text">本章主要讲解异常和中断。第一部分主要讲解了中断程序的寻找过程：从CPU给出指令号，到计算此异常指令向量表地址，取相应地址，跳转内存对应地址执行这一过程。因为不同时代的CPU实现此过程的模式不同，对于实模式，其采用寄存器保存中断向量表，CS和IP的结构为4字节，对于保护模式，其采用储存器保存中断向量表，对于一个中断标志号，需要根据IDTR找到偏移值，根据CS和GDTR找到基地址，然后算出真实物理地址，进行指令执行。</p><p type="card-text">在第二部分，主要讲解了CPU执行中断的整体流程，包括其对于当前指令和数据的处理、对指令和数据的恢复，重点讲解了Flag寄存器的作用，中断返回的过程。之后，讲解了不同的中断指令，包括内部和外部中断，以及其优先级和指令、使用到的FLAGS等。最后，讲解了BIOS和DOS利用中断进行丰富功能的实现的操作。</p></span>

# 中断和异常的来源

- 第一个带有外部中断的系统：DYSEAC，1954年
    - 有两个程序计数器（PC），根据I/O信号进行切换
    
- 第一个带有异常处理的系统：UNIVAC，1951年
    - 当ALU运算出错，直接连线到PC，更新地址到00
    - 算术运算溢出时：转向地址0执行两条修复指令，或者停机
    - 1955年，UNIVAC 1103 增加了外部中断，用于风洞数据的实时收集
    
简要来说，将计算机的异常分为内部中断和外部中断，其中内部中断指的是因为ALU运行出错（比如结果溢出）导致的，而外部中断指的是I/O设备导致的（比如需要输入输出）。

# 中断向量表的结构

## Intel 8086

存储器中保留两个专用区域
- 中断向量表区：00000H~003FFH（地址最低的1KB）
- 初始化程序区：FFFF0H~FFFFFH（地址最高的16B）



![](w8p1.png)

如上图所示，在8086的寄存器中的底端保存着中断向量表，这个表保存着对应错误指令对应的处理地址，其占有256*4个字节，可以处理256种错误。每个错误对应的地址有4个字节。

其中前两个字节保存着中断服务程序入口地址的偏移量（IP），低字节在前，高字节在后。后两个字节保存段基值（CS），其中低字节在前，高字节在后。

比如 06H 30H 00H 40H （由左到右为IP低，IP高，CS低，CS高）可以组成 4000H 3006H （分别对应CS,IP），合并起来就是 40003006H 这个物理地址。

![](w8p2.png)

因此，如果CPU现在有一个称之为20H的中断类型码，也就是对应十进制32，32*4 = 128（因为每四个字节对应1个向量）， 转换为十六进制是 80H。因此，其起始地址为0000：0080H。

当CPU从这个寄存器获取到中断向量编号地址的时候，找到这四个字节，进行段基址和偏移地址的加法，得到存储器地址，到存储器中获取中断对应的服务程序。在存储器中的中断服务程序可以不按顺序编排，只要对应中断向量表中的值即可。

若中断类型码为17H，中断服务程序的入口地址为 2340H:7890H，则地址由低到高分别为____，对应的内容分别为 ____。

0000:005CH, 0000:005DH,
0000:005EH, 0000:005FH

90H, 78H, 40H, 23H

其中8086的中断向量表中，类型0-4对应除法错、单步、非屏蔽、断点、溢出，而类型5-31则表示保留的中断，类型32到255是供用户定义的中断。

![](w8p3.png)

80386对应的中断向量表如上所示，这个表一直持续到Core2。

## IA-32

IA-32提供了实模式和保护模式，对于实模式，其指令的寻址还是由 `CS:IP` 构成，而保护模式的地址则是由 `CS:EIP` 构成。EIP包含2**32=4G字节的单元，而传统IP包含2**16=65k字节的单元。

在保护模式下，段基址不在CS中，而是在内存中，GDTR全局描述符表的地址寄存器包含着这一内存地址的低地址标记。CS依然被使用，其包含了8个字节的内容个，如下，不过，CS+GDTR提供的低地址才能正确找到对应的描述符，取出基地址。

![](w8p4.png)


![](w8p5.png)

当CPU遇到一个中断的时候，其会根据`中断类型码*8+IDTR(中断描述符表地址寄存器)`中提供的地址来形成IP，由SDTR和CS形成的段基址合并生成中断服务程序的真实物理地址。

# 中断处理过程

1. 关中断
    - CPU关闭中断响应，即不再接受其它外部中断请求
2. 保存断点
    - 将发生中断处的指令地址压入堆栈，以使中断处理完后能正确地返回
3. 识别中断源
    - CPU识别中断的来源，确定中断类型号，从而找到相应的中断服务程序的入口地址
4. 保护现场
    - 将发生中断处的有关寄存器（中断服务程序中要使用的寄存器）以及标志寄存器的内容压入堆栈
5. 执行中断服务程序
    - 转到中断服务程序入口开始执行，可在适当时刻重新开放中断，以便允许响应较高优先级的外部中断
6. 恢复现场并返回
    - 把“保护现场”时压入堆栈的信息弹回原寄存器，然后执行中断返回指令，从而返回主程序继续运行

## IF标志符

软件开放和关闭中断响应地方法：标志寄存器（FLAGS）中的**IF标志位中是中断标志，DF为方向标志，TF为跟踪标志**。

如果IF=1，那么允许CPU相应可屏蔽中断请求，如果IF=0，那么不允许CPU相应可频闭中断请求。STI用来将IF置为1，CLI用来清零，需要注意，IF对于内部中断和非屏蔽中断不起作用，仅对于特定外部中断可以屏蔽。

## 从中断中返回

IRET指令（中断返回）
- 格式：IRET
- 操作
    - 从栈顶弹出3个字，分别送入IP、CS和FLAGS寄存器
- 说明
    - 放在中断服务程序的末尾
    - 按中断调用时的逆序恢复现场
    - 返回到程序发生中断处继续执行

扩展：IRETD指令， IRETQ指令

总的流程如图所示，需要注意，在步骤7前需要进行数据现场保存(4,5)，在数据10之前逆序恢复（9）。

![](w8p6.png)

以下是X86内部和外部中断的类别。

![](w8p7.png)

# 内部中断分类

## 除法错中断

在执行除法指令后，若所得的商超出了目标寄存器所能表示的范围，比如用数值0作除数，则CPU立即产生一个类型0中断。

## 溢出中断

- 执行INTO指令时，若溢出标志位OF为1，则将引起类型为4的内部中断
- 执行INTO指令时，若溢出标志位OF为0，则INTO指令执行空操作
- INTO指令通常安排在算术运算指令之后，以便在发生溢出时能及时处理
- 指令INTO等同于指令INT 4

## 单步中断

- 当标志寄存器的TF位置1之后，CPU便处于单步工作方式
- 在单步工作方式下，CPU每执行完一条指令，就会自动产生一个类型1中断，进入类型1中断服务程序
- 类型1中断服务程序：一般用于显示CPU内部各寄存器的内容和一些其它信息，以便进行调试和发现错误

## 断点中断

- 与单步中断类似，断点中断也是一种调试程序的手段，并且常常和单步中断结合使用
- 对一个大的程序，一般先通过断点将程序中的某一错误确定在程序的一小段，再对这一小段程序用单步方式跟踪调试
- 在所有INT n形式的指令中，只有断点中断指令INT 3是一条单字节长的指令，其它都是两字节指令
- INT 3的指令编码：11001100
- 断点中断会在中断处设置INT 3指令，之后当程序运行到断点，进入中断服务程序，显示CPU各寄存器的值等，之后恢复用户程序原有指令，IP-1。CPU从断点处继续执行。

## 注意事项

除单步中断外，所有内部中断都不可以用软件方法来禁止（屏蔽），单步中断可通过软件将TF标志置1或清0来允许或禁止。

除单步中断外，所有内部中断的优先级都比外部中断高

内部中断的类型号由CPU内部产生，外部中断则需要从外设读取中断类型号

# 基于中断的功能调用

`INT n` x86根据此指令调用中断服务程序，其中n为0-255的任意数，对应中断类型码。

操作：

① 将FLAGS寄存器的内容压栈

② 清除中断标志IF和单步标志TF

③ 将CS和IP寄存器的内容压栈

④ 根据中断类型码查找中断向量表，取得对应中断服务程序的入口地址

⑤ 将入口地址分别装入CS和IP寄存器

## BIOS中断

ROM BIOS(Basic Input Output System)

◦ 装于从地址0FE00H开始的8KB ROM中

◦ 提供了系统加电自检、引导装入、主要I/O设备的处理程序及接口控制等功能模块

BIOS中断

◦ BIOS各功能模块的入口地址都在中断向量表中

◦ 通过软件中断指令“INT n”调用各功能模块

◦ 如有需要，使用寄存器传递参数

![](w8p8.png)

## DOS中断

同样的，DOS中断可以用来指定更加丰富的操作。

格式：INT 21H

功能

- 包含最常用的功能程序，分别实现文件管理、存储管理、作业管理和设备管理等功能

- 共用21H号中断入口，通过传参数的方式设置功能号，以选择执行不同功能模块的代码

说明

- DOS中断功能比BIOS中断更齐全、完整

- 进一步屏蔽了设备的物理特性及其接口特性

![](w8p9.png)